# Useful Functions for Working with Train DNA

A file to call functions from for working with the TrainDNA dataset. This keeps all of the files consistent across the notebooks, and allows them to be updated easily. **DO NOT CHANGE THEM WITHOUT THOUGHT PLEASE!!!**

In [1]:
from neuro_python.neuro_compute import spark_manager as spm
import neuro_python.neuro_compute.notebook_manager as notebook_manager
from neuro_python.neuro_data import schema_manager as sm
from neuro_python.neuro_data import sql_query as sq
from neuro_python.neuro_data import sql_commands as sc
import plotly.plotly as py
import plotly.graph_objs as pgo
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import ipywidgets as widgets
import datetime as dt

In [2]:
spm.create_context('TrainDNAFunctionsHB')

{'ContextId': 'ab2bf544-b175-44bb-940e-7d02c877b6fb',
 'IdleContextLifeInMinutes': 20}

## Success Starts

This function finds the number of true starts for a dataset, by filtering out the non-CPG Startup Signals, and then classifying correct starts as when there is more than 30 min between start events. If the events occur closer than that, then the Train is assumed to have not started correctly i.e. startTimes = [10:00, 10:01, 18:00] then successful start times = [10:01, 18:00]

In [3]:
%%spark 
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf,PandasUDFType

schema2 = StructType([
    StructField("CSN", StringType()),
    StructField("EventID", LongType()),
    StructField("DateTime", StringType())
])

 

@pandas_udf(schema2, functionType=PandasUDFType.GROUPED_MAP)
def startTimes(df):
    gr = df['CSN'].iloc[0]#used for the default returned dataframe if no data being returned
    EventID = 65533
    events = df[df['EventID']==EventID].sort_values(['DateTime']) #start events
    DateTime = pd.to_numeric(pd.to_datetime(events['DateTime']))/1e9 #convert ns to s
    DateTimeNext = DateTime.shift(periods=-1) # -1 because we want to find the gap until the NEXT one (ie. if THIS event failed)
    time_diff = (DateTimeNext-DateTime) #Find the difference in event times
    df_less15m = events[((time_diff>(30*60)) | (time_diff.isnull()))]#more than 30 min or when time_diff is null, as that is the last recorded event
    default_df = pd.DataFrame([[gr,EventID,'None']]) #Returned if no start events detected - filter out using Datetime == none
    if len(df_less15m.index)>0:
        return_df = pd.DataFrame(df_less15m[['CSN','EventID','DateTime']])
        #reset column names, this needs to be done otherwise it won't match the schema above. It works when we do .mean() etc
        #because that doesn't give a named column when we create the dataframe, whereas this does.
        return_df = return_df.T.reset_index(drop=True).T #this feels hacky
        return return_df
    else:
        return default_df

VBox(children=(Output(), HBox(children=(Label(value='CommandId: 19702d50-c75a-441e-8eb1-1653534ad609'), FloatProgress(value=0.0, max=1.0), Button(description='Cancel', style=ButtonStyle())))))

## Fail Starts

This one finds the starts which are closer than 30 minutes but further than 10 seconds apart. This means that it is not a repeat signal recorded or the operator pressing the button multiple times, and not a successful start. I.e. start times = [10:00:00, 10:00:03, 10:01:00, 12:00:30] returns 10:00:03 as a failed event


In [4]:
%%spark 
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf,PandasUDFType

schema3 = StructType([
    StructField("CSN", StringType()),
    StructField("EventID", IntegerType()),
    StructField("DateTime", StringType())
])

@pandas_udf(schema3, functionType=PandasUDFType.GROUPED_MAP)
def failTimes(df):
    gr = df['CSN'].iloc[0]#used for the default returned dataframe if no data being returned
    EventID = 65533
    events = df[df['EventID']==EventID].sort_values(['DateTime'])
    DateTime = pd.to_numeric(pd.to_datetime(events['DateTime']))/1e9 #convert ns to s
    DateTimeNext = DateTime.shift(periods=-1) # -1 because we want to find the gap until the NEXT one (ie. if THIS event failed)
    time_diff = (DateTimeNext-DateTime)
    df_less15m = events[((time_diff<(30*60)) & (time_diff > 10))]#more than 10s, less than 30 mins
    default_df = pd.DataFrame([[gr,EventID,'None']])
    if len(df_less15m.index)>0:
        return_df = pd.DataFrame(df_less15m[['CSN','EventID','DateTime']])
        #reset column names, this needs to be done otherwise it won't match the schema above. It works when we do .mean() etc
        #because that doesn't give a named column when we create the dataframe, whereas this does.
        return_df = return_df.T.reset_index(drop=True).T #this feels hacky
        return return_df
    else:
        return default_df
    

VBox(children=(Output(), HBox(children=(Label(value='CommandId: 16e57475-092f-4ff3-9f2a-83da8b290fc7'), FloatProgress(value=0.0, max=1.0), Button(description='Cancel', style=ButtonStyle())))))

## Remove duplicate signals

This UDF removes sigals which have the same CSN and Event ID which occur within 1 second of each other, as it assumes that are a duplicate alert i.e. signals for one CSN of the same Event ID occured at [10:00:00:123, 10:00:00:956, 10:00:01:342] returns [10:00:00, 10:00:01]

In [5]:
%%spark 
#Run udf on octSelectEvents to remove signals with same second values
#i.e same eventID time =10:00:01 and again with time = 10:00:01 then remove one
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf,PandasUDFType

schema5 = StructType([
    StructField("CSN", StringType()),
    StructField("EventID", IntegerType()),
    StructField("EventIDNext", IntegerType()),
    StructField("DateTime", StringType()),
    StructField("DateTimeInt", IntegerType()),
    StructField("DateTimeIntNext", IntegerType()),
    StructField("timeDiff", IntegerType())
])

@pandas_udf(schema5, functionType=PandasUDFType.GROUPED_MAP)
def removeDuplicates(df):
    df.rename(columns=lambda x: x.lstrip(), inplace = True)
    
    gr = df['CSN'].iloc[0]#used for the default returned dataframe if no data being returned
    EventID = 65533
    df['DateTimeInt'] = pd.to_numeric(pd.to_datetime(df['DateTime']))/1e9 #convert ns to s
    df = df.sort_values(['EventID','DateTimeInt'])

    df['DateTimeIntNext'] = df['DateTimeInt'].shift(periods=-1) # -1 because we want to find the gap until the NEXT one (ie. if THIS event failed)
    df['EventIDNext'] = df['EventID'].shift(periods=-1)
    
    df['timeDiff'] = (df['DateTimeIntNext']-df['DateTimeInt'])
    
   #Remove the rows with timeDiff between -1 and 1 and the same event ID
    dResult = df[((df.timeDiff < -1) | (df.timeDiff > 1)) | (df.EventID != df.EventIDNext)]
    
    #If empty array: i.e. only one event type happens, all at the same time, return the first row
    if dResult.empty:
        dResult = df.iloc[[0]]
    return dResult

VBox(children=(Output(), HBox(children=(Label(value='CommandId: f10cd0d8-3550-4b16-a849-dd153e529f01'), FloatProgress(value=0.0, max=1.0), Button(description='Cancel', style=ButtonStyle())))))

## Get time relative to the closest event of a chosen event type - i.e. Start up

This UDF returns the time between each event of any ID and the nearest event of a chosen event ID, with positive or negative seconds indicating after or before the event occuring. This can be used to analyse Train behaviour around a given event, for example the start up, or shut down of a train. Filtering using removeDuplicates or successStarts is reccomended to remove some of the events that wish to be analysed. The event to analyse is declared in startID

In [6]:
%%spark 
import pandas as pd
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf,PandasUDFType

schema4 = StructType([
    StructField("CSN", StringType()),
    StructField("EventID", IntegerType()),
    StructField("DateTime", StringType()), 
    StructField("TimeToStart", IntegerType()),
    StructField("DateTimeInt", IntegerType()),
    StructField("startIndex", IntegerType())
])

@pandas_udf(schema4, functionType=PandasUDFType.GROUPED_MAP)
def getClosestStartsandTimes(df):
    df.rename(columns=lambda x: x.lstrip(), inplace = True)
    gr = df['CSN'].iloc[0]#used for the default returned dataframe if no data being returned
    
    #Get time in seconds
    df['DateTimeInt'] = pd.to_numeric(pd.to_datetime(df['DateTime']))/1e9 #convert ns to s
    
    #Get Start Event times - startID can be changed for any event to anaylse train behaviour around that event
    startID = 65533
    startEvents = df[df['EventID']==startID]
    
    #if no start events for CSN, return empty array, otherwise perfrom calcs - reccommend that a check is performed after the calling of this function to check if any Datetime is listed as 'WRONG:
    if startEvents.empty:
        otherEvents = pd.DataFrame([[gr,65533,'WRONG',1,2,3]])
    else:#Otherwise...
        otherEvents = df[df['EventID']!=startID] #Remove the start events
        st = startEvents['DateTimeInt'].values #outputs a list
        ot = otherEvents['DateTimeInt'].values
        
        #put into a numpy array to make use of numpy functions
        stNp = np.array(st) #stNp is a list of the times of the start events for the time period - compare time of each event to these vals
        otNp = np.array(ot)
        
        #Get shape of df to find no of rows
        col = len(ot)
        ot = ot.reshape(col,1)

        #Add start times to each row of dataframe
        stList = stNp #Declaring the list to be appending stNp to
        for i in range(col-1):
            stList = np.vstack((stList,stNp))
        
        #compute the time diff for each start time
        timeDiff = ot - stList
        
        #Find the minimum time value, and the index which it occurs at
        x = np.argmin(abs(timeDiff),axis=1).reshape(col,1)
        otherEvents['startIndex'] = x
        
        #Store the minimum time in the correct rows
        timeDiffResult = []
        for i in range(col):
            val = (timeDiff[i,x[i]])
            timeDiffResult.append(val[0])

        #Make new numpy of timeDiffResult and get min - NOT USED ANY MORE
        minTime = np.argmin(abs(np.array(timeDiffResult)))

        #Make new col with the time ebtween said event and the nearest start Time
        otherEvents['TimeToStart'] = (timeDiffResult)


    return otherEvents

VBox(children=(Output(), HBox(children=(Label(value='CommandId: 2566b8dd-c819-4ca8-8b22-8dcfcbbff444'), FloatProgress(value=0.0, max=1.0), Button(description='Cancel', style=ButtonStyle())))))